### Select a sample of biotech and pharma public companies

| Ticker | Company Name               | Platform / Focus                                            | Development Stage                                                      |
|:------:|---------------------------|-------------------------------------------------------------|-------------------------------------------------------------------------|
| VERV   | Verve Therapeutics         | Base editing (in vivo CRISPR) targeting PCSK9/Lipoproteins | Early clinical (Phase 1b “Heart-2”); Fast Track; lead candidate VERVE‑102 |
| NTLA   | Intellia Therapeutics      | CRISPR/Cas9 — both in vivo (liver, ATTR/HAE) & ex vivo cell editing | Clinical-stage: Phase 1/2 for ATTR and HAE; recently dosed Phase 3 patient |
| EDIT   | Editas Medicine           | CRISPR/Cas9 & Cas12a for in vivo (HSC, liver) and ex vivo applications | Preclinical & early clinical—proof-of-concept in vivo data presented |
| BIIB   | Biogen                     | Neurology & rare disease biologics (MS, ALS, spinal muscular atrophy) | Late-stage clinical and commercial; multiple Phase 2/3 neuroimmunology candidates |

In [60]:
import os
import re
import json
import requests
import pandas as pd
from typing import Dict, List
from datetime import datetime
from collections import defaultdict

# Load API Key
CAPTIDE_API_KEY = os.getenv("CAPTIDE_API_KEY")

# Set headers
HEADERS = {
    "X-API-Key": CAPTIDE_API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Define tickers and cutoff date
TICKERS = ["VERV", "NTLA", "EDIT", "BIIB"]
cutoff_date = "2022-12-31"

### Select Relevant Documents

| Event Type                             | Typical 8-K Item | Notes                                           |
|---------------------------------------|------------------|-------------------------------------------------|
| Clinical trial results (early or final) | 7.01 or 8.01     | Often voluntary under FD or Other Events        |
| FDA approval or rejection              | 7.01 or 8.01     | BLA/NDA/IND status updates                      |
| Clinical hold or warning letter        | 8.01             | Sometimes also 7.01 if disclosed widely         |
| Program discontinuation                | 8.01 or 2.05     | 2.05 if cost-related                            |
| New collaboration or licensing deal    | 1.01             | Often with financial terms                      |
| Termination of a partnership/license   | 1.02             | Could signal clinical failure                   |

In [61]:
def is_valid_document(doc: Dict) -> bool:
    if doc["sourceType"] == "8-K":
        item_str = doc.get("additionalKwargs", {}).get("item", "")
        relevant_items = {"1.01", "1.02", "2.05", "7.01", "8.01"}
        return any(item in item_str for item in relevant_items)
    return True

def parse_sse_response(sse_text: str) -> Dict:
    try:
        lines = [l[6:] for l in sse_text.splitlines() if l.startswith("data: ")]
        for l in lines:
            obj = json.loads(l)
            if obj.get("type") == "full_answer":
                content = re.sub(r"\s*\[#\w+\]", "", obj["content"])
                m = re.search(r"\{.*\}", content, re.DOTALL)
                return json.loads(m.group(0)) if m else {}
    except Exception:
        pass
    return {}

In [62]:
def fetch_documents(ticker: str) -> List[Dict]:
    url = f"https://rest-api.captide.co/api/v1/companies/ticker/{ticker}/documents?startDate={cutoff_date}"
    response = requests.get(url, headers=HEADERS, timeout=60)
    docs = response.json()
    print(f"Fetched {len(docs)} raw documents for {ticker}")
    return [
        {
            "ticker": doc["ticker"],
            "sourceLink": doc["sourceLink"],
            "date": doc["date"]
        }
        for doc in docs
        if doc["sourceType"] == "8-K"
        and "fiscalPeriod" in doc
        and is_valid_document(doc)
    ]

# Example:
test_docs = fetch_documents("VERV")
print(json.dumps(test_docs, indent=2))

Fetched 30 raw documents for VERV
[
  {
    "ticker": "VERV",
    "sourceLink": "https://rest-api.captide.co/api/v1/document?sourceType=8-K&documentId=6eff02cb-497d-466c-bf9e-54f5e29b2777",
    "date": "2025-06-17"
  },
  {
    "ticker": "VERV",
    "sourceLink": "https://rest-api.captide.co/api/v1/document?sourceType=8-K&documentId=0a60ebbd-f8cb-4d1b-a95f-d132e83d4a2d",
    "date": "2025-06-17"
  },
  {
    "ticker": "VERV",
    "sourceLink": "https://rest-api.captide.co/api/v1/document?sourceType=8-K&documentId=59e6ea4b-3806-4d72-8c2b-a0cd2dd8db51",
    "date": "2025-06-17"
  },
  {
    "ticker": "VERV",
    "sourceLink": "https://rest-api.captide.co/api/v1/document?sourceType=8-K&documentId=32f5d6ca-c90a-4f7b-b45d-2698c16d5479",
    "date": "2025-06-17"
  },
  {
    "ticker": "VERV",
    "sourceLink": "https://rest-api.captide.co/api/v1/document?sourceType=8-K&documentId=42e15405-bf9a-407b-add4-f734c77588d4",
    "date": "2025-06-17"
  },
  {
    "ticker": "VERV",
    "sourceLink": 

In [63]:
def fetch_metrics_with_prompt(source_links: List[str], prompt: str) -> Dict:
    payload = {"query": prompt, "sourceLink": source_links}
    r = requests.post(
        "https://rest-api.captide.co/api/v1/rag/agent-query-stream",
        json=payload, headers=HEADERS, timeout=120
    )
    return parse_sse_response(r.text)

BASE_PROMPT = (
    """
    Return a single valid JSON object with the following key-value pairs. Note that "eventType" dictates the required structure of the "eventDetails" field:

    - "eventType": one of the following values — "clinical_results", "fda_decision", "clinical_hold", "program_termination", or "program_announcement".

    - "eventDetails": an object containing structured details relevant to the specified "eventType", using the required fields listed below.

    If the document does **not** contain any material information matching these event types, return:
    {
    "eventType": null,
    "eventDetails": {}
    }

    Required fields per event type:

    - If "eventType" = "clinical_results":
    - "trialPhase" — e.g., Phase 1/2/3
    - "drugName" — name of the investigational therapy
    - "indication" — disease or condition being treated
    - "resultsSummary" — concise summary of efficacy outcomes
    - "safetyProfile" — any adverse events or tolerability findings
    - "nextSteps" — upcoming plans (e.g., advancing to next phase)

    - If "eventType" = "fda_decision":
    - "decisionType" — e.g., approved, Complete Response Letter (CRL), delayed
    - "applicationType" — e.g., NDA (New Drug Application), BLA (Biologics License Application)
    - "drugName" — name of the drug reviewed
    - "indication" — condition the drug is intended to treat
    - "notes" — relevant regulatory context, labeling info, or committee outcomes

    - If "eventType" = "clinical_hold":
    - "holdType" — full or partial hold
    - "reason" — FDA's stated rationale for the hold
    - "drugName" — drug or program affected
    - "impactSummary" — brief on trial or pipeline impact
    - "resolutionPlan" — company's steps to address the hold

    - If "eventType" = "program_termination":
    - "programName" — name of the discontinued program
    - "indication" — target disease area
    - "developmentStage" — how far along the program was (e.g., preclinical, Phase 2)
    - "reason" — scientific, strategic, or commercial rationale for termination
    - "financialImpact" — expected cost savings or write-downs
    - "strategicNotes" — effect on company direction or pipeline focus

    - If "eventType" = "program_announcement":
    - "programName" — name of the new or updated program
    - "indication" — disease or condition being targeted
    - "platform" — underlying technology (e.g., CRISPR, base editing)
    - "developmentStage" — current status (e.g., preclinical, IND-enabling)
    - "milestone" — key progress point announced (e.g., IND submission, trial start)
    - "partnershipInfo" — details of any collaborations or licensing
    - "timelineEstimate" — estimated timing of next milestone
    """
)

In [64]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_ticker(ticker: str):
    try:
        documents = fetch_documents(ticker)
        table_data = defaultdict(lambda: defaultdict(str))  # eventType -> year -> detail
        for doc in documents:
            year = datetime.strptime(doc["date"], "%Y-%m-%d").year
            parsed = fetch_metrics_with_prompt([doc["sourceLink"]], BASE_PROMPT)
            event_type = parsed.get("eventType")
            if event_type:
                full_event = {"eventType": event_type, "eventDetails": parsed["eventDetails"]}
                table_data[event_type][year] += json.dumps(full_event) + "\n---\n"
        if table_data:
            df = pd.DataFrame(table_data).T
            return ticker, df
        else:
            return ticker, None
    except Exception as e:
        return ticker, None

results_by_ticker = {}

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(process_ticker, ticker): ticker for ticker in TICKERS}
    for future in as_completed(futures):
        ticker, df = future.result()
        if df is not None:
            results_by_ticker[ticker] = df

Fetched 30 raw documents for VERV
Fetched 50 raw documents for NTLA
Fetched 48 raw documents for EDIT
Fetched 48 raw documents for BIIB


In [65]:
def camel_to_title(s):
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', s).replace("_", " ").title()

for ticker, df in results_by_ticker.items():
    print(f"\n📢 Ticker: {ticker}")
    
    for event_type, row in df.iterrows():
        for year, detail in row.items():
            if not isinstance(detail, str) or not detail.strip():
                continue

            print(f"\n🗂️ {ticker} — {event_type} — {year}")

            try:
                # Split by delimiter and parse each event separately
                event_blobs = [blob for blob in detail.split("\n---\n") if blob.strip()]
                for blob in event_blobs:
                    parsed_detail = json.loads(blob)
                    details = parsed_detail.get("eventDetails", {})

                    if isinstance(details, dict) and details:
                        display(pd.DataFrame.from_dict(
                            {camel_to_title(k): v for k, v in details.items()},
                            orient="index",
                            columns=["Value"]
                        ).style.set_properties(**{'white-space': 'pre-wrap'}))
                    else:
                        print(f"⚠️ No eventDetails in blob: {blob}")
            except json.JSONDecodeError as e:
                print(f"❌ JSON decode error for {event_type} in {year}: {e}")
            except Exception as e:
                print(f"❌ Failed to parse/display event for {event_type} in {year}: {e}")



📢 Ticker: VERV

🗂️ VERV — program_announcement — 2025


,Value
Program Name,VERVE-102
Indication,"Atherosclerotic cardiovascular disease (ASCVD), including heterozygous familial hypercholesterolemia (HeFH) and certain patients with premature coronary artery disease (CAD)"
Platform,In vivo gene editing
Development Stage,Phase 1b clinical trial
Milestone,Granted Fast Track designation by the U.S. Food and Drug Administration; ongoing Phase 1b clinical trial
Partnership Info,"Verve Therapeutics to be acquired by Eli Lilly and Company, with Lilly to accelerate development and commercialization of Verve's gene editing programs"
Timeline Estimate,Transaction expected to close in Q3 2025; contingent value right (CVR) payable upon first patient dosed with VERVE-102 in a U.S. Phase 3 clinical trial within ten years of closing



📢 Ticker: BIIB

🗂️ BIIB — fda_decision — 2023


,Value
Decision Type,approved
Application Type,NDA
Drug Name,SKYCLARYS® (omaveloxolone)
Indication,Friedreich’s ataxia
Notes,"SKYCLARYS® is the first and only FDA approved treatment for Friedreich’s ataxia in the U.S. The commercial launch is underway, and over 1,000 patient start forms have been submitted. European regulatory review is ongoing, and the drug has Orphan Drug designation in Europe."


,Value
Decision Type,approved
Application Type,NDA
Drug Name,SKYCLARYS (omaveloxolone)
Indication,Friedreich’s ataxia in adults and adolescents aged 16 years and older
Notes,SKYCLARYS (omaveloxolone) received FDA approval for the treatment of Friedreich’s ataxia in the U.S. The drug is also under review by the European Medicines Agency (EMA) and has received Orphan Drug designation in Europe for the same indication.


...

,Value
Trial Phase,Phase 1/2/3
Drug Name,renizgamglogene autogedtemcel (reni-cel; formerly EDIT-301)
Indication,severe sickle cell disease (SCD)
Results Summary,"In the RUBY trial (N=28), 27 of 28 patients treated with reni-cel were free of vaso-occlusive events (VOEs) post-infusion. Patients showed early normalization of total hemoglobin (mean increased from 9.8 g/dL at baseline to 13.8 g/dL at Month 6) and rapid, sustained improvements in fetal hemoglobin (mean HbF 48.1% at Month 6, n=18). The mean percentage of F-cells was sustained at >90% from month 4 onward, and markers of hemolysis improved or normalized by Month 6. Clinically meaningful improvements were observed in pain, physical function, and social roles per patient-reported outcomes."
Safety Profile,Reni-cel was well-tolerated with a safety profile consistent with myeloablative busulfan conditioning and autologous hematopoietic stem cell transplant. All evaluable patients (n=27) achieved successful engraftment (median neutrophil engraftment: 23 days; platelet: 25 days). Two serious adverse events possibly related to reni-cel were reported.
Next Steps,"The RUBY trial is ongoing, with continued follow-up of patients. Additional details are available on clinicaltrials.gov (NCT04853576)."


,Value
Trial Phase,Phase 1/2/3
Drug Name,renizgamglogene autogedtemcel (reni-cel; formerly known as EDIT-301)
Indication,severe sickle cell disease
Results Summary,"In the RUBY trial, all 18 patients treated with reni-cel were free of vaso-occlusive events (VOEs) for up to 22.8 months of follow-up. Patients experienced early normalization of total hemoglobin (mean >14 g/dL) and rapid, sustained increases in fetal hemoglobin (mean HbF >40%). At 6 months, mean total Hb was 14.3 g/dL and mean HbF was 48.5%. The mean percentage of F-cells was sustained at >90% from month 4 onward, and mean corpuscular fetal hemoglobin (MCH-F) was above the anti-sickling threshold of 10 pg/F-cell by month 3. Markers of hemolysis were normalized or improved, and all patients showed sustained high levels of editing in the HBG1 and HBG2 promoter regions."
Safety Profile,"Reni-cel was well-tolerated with a safety profile consistent with myeloablative conditioning with busulfan and autologous hematopoietic stem cell transplant. All patients demonstrated successful neutrophil and platelet engraftment (median 23 and 24 days, respectively). No serious adverse events related to reni-cel treatment were reported."
Next Steps,Editas Medicine has completed adult cohort enrollment and opened the adolescent cohort in the RUBY trial. Additional clinical data from the RUBY and EdiTHAL trials are expected to be presented by year-end 2024.


,Value
Trial Phase,Phase 1/2
Drug Name,renizgamglogene autogedtemcel (reni-cel; formerly EDIT-301)
Indication,transfusion-dependent beta thalassemia (TDT)
Results Summary,"All 7 patients treated in the EdiTHAL trial maintained hemoglobin levels above the transfusion threshold and are transfusion-free post-reni-cel infusion. For patients with ≥6 months follow-up, mean total hemoglobin at month 6 was 12.1 g/dL (SD 1.3) and mean fetal hemoglobin was 10.9 g/dL (SD 1.5), sustained at or above the transfusion threshold from 6 months onward. All patients showed sustained high levels of editing in the HBG1 and HBG2 promoter regions."
Safety Profile,Reni-cel was well-tolerated with a safety profile consistent with myeloablative conditioning with busulfan and autologous hematopoietic stem cell transplant. All patients demonstrated successful neutrophil (median 23 days) and platelet (median 38 days) engraftment. No serious adverse events related to reni-cel treatment were reported.
Next Steps,Editas Medicine will continue to evaluate the effectiveness of reni-cel in the EdiTHAL trial and plans to present further clinical updates from both the EdiTHAL and RUBY trials by year-end 2024.



🗂️ EDIT — program_termination — 2024


,Value
Program Name,reni-cel
Indication,sickle cell disease and beta thalassemia
Development Stage,clinical (RUBY and EdiTHAL trials)
Reason,Extensive search did not yield a commercial partner
Financial Impact,"Initiating cost savings measures and reduction in headcount to align workforce and resources to in vivo pipeline, extending cash runway into Q2 2027"
Strategic Notes,"Editas Medicine is transitioning to a fully in vivo gene editing company, focusing on in vivo CRISPR-edited medicines and discontinuing ex vivo programs like reni-cel. The company will work with clinical trial sites, regulators, and other parties to determine the path forward for patients enrolled in the RUBY and EdiTHAL trials."
